In [2]:
# TODO : Normalization
# TODO : add z
# TODO : CTC Loss
from typing import List

import cv2
import gdown
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Compose, Lambda

#import tensorflow as tf

In [41]:
# !jupyter nbconvert lip_keypoint.ipynb --to python

[NbConvertApp] Converting notebook lip_keypoint.ipynb to python
[NbConvertApp] Writing 8638 bytes to lip_keypoint.py


In [1]:
df3 = pd.read_csv("face_landmarks_data3.csv", index_col=0)
df4 = pd.read_csv("face_landmarks_data4.csv", index_col=0)
df5 = pd.read_csv("face_landmarks_data5.csv", index_col=0)
df6 = pd.read_csv("face_landmarks_data6.csv", index_col=0)
df = pd.concat([df3, df4, df5, df6], ignore_index=True)

NameError: name 'pd' is not defined

In [55]:
video_path_counts = df['video_path'].value_counts()

video_paths_to_keep = video_path_counts[video_path_counts == 3000].index

df = df[df['video_path'].isin(video_paths_to_keep)]

In [58]:
mean_x = np.mean(df['x'])
std_x = np.std(df['x'])
mean_y = np.mean(df['y'])
std_y = np.std(df['y'])
mean_z = np.mean(df['z'])
std_z = np.std(df['z'])

# df['x'] = (df['x'] - mean_x) / std_x
# df['y'] = (df['y'] - mean_y) / std_y
# df['z'] = (df['z'] - mean_z) / std_z

In [62]:
# df = df[:4000]

In [63]:
df.head(5)

,frame,x,y,z,visibility,video_path
0,1,0.363994,0.539015,-1.724262,0.0,data2/s3\bbaf1s.mpg
1,1,0.779797,0.458158,-1.623525,0.0,data2/s3\bbaf1s.mpg
2,1,1.206504,0.632151,-0.997362,0.0,data2/s3\bbaf1s.mpg
3,1,1.496412,0.851839,-0.201793,0.0,data2/s3\bbaf1s.mpg
4,1,0.362648,1.066710,-0.628563,0.0,data2/s3\bbaf1s.mpg


In [64]:
df.groupby('video_path').count()

,frame,x,y,z,visibility
video_path,,,,,
data2/s3\bbaf1s.mpg,3000,3000,3000,3000,3000
data2/s3\bbaf2p.mpg,3000,3000,3000,3000,3000
data2/s3\bbaf3a.mpg,3000,3000,3000,3000,3000
data2/s3\bbafzn.mpg,3000,3000,3000,3000,3000
data2/s3\bbal4n.mpg,3000,3000,3000,3000,3000
...,...,...,...,...,...
data2/s3\swwpzn.mpg,3000,3000,3000,3000,3000
data2/s3\swwv4n.mpg,3000,3000,3000,3000,3000
data2/s3\swwv5s.mpg,3000,3000,3000,3000,3000


In [6]:
df_filtered = df.copy()
chunks = []
num_frames_per_chunk = 3000
for video_path, group in df_filtered.groupby('video_path'):
    num_frames = len(group)
    num_chunks = num_frames // num_frames_per_chunk
    for i in range(num_chunks):
        chunk = group.iloc[i*num_frames_per_chunk:(i+1)*num_frames_per_chunk]
        chunk_reshaped = chunk[['x', 'y', 'z']].values.reshape(-1, 75, 40*3)
        chunks.append(chunk_reshaped)

input_data = np.concatenate(chunks, axis=0)

In [7]:
input_data.shape

(986, 75, 120)

In [8]:
def create_vocab():
    vocab = "abcdefghijklmnopqrstuvwxyz123456789 "
    return vocab


def char_to_int(char):
    # shift 1 
    vocab = "abcdefghijklmnopqrstuvwxyz123456789 "
    return vocab.index(char) + 1 if char in vocab else -1

def int_to_char(index):
    # shift 1 
    vocab = "abcdefghijklmnopqrstuvwxyz123456789 "
    return vocab[index - 1] if 1 <= index <= len(vocab) else ''

def load_alignments(path:str) -> List[str]:
    with open(path, 'r') as f:
        lines = f.readlines()
    tokens = []
    for line in lines:
        line = line.split()
        if line[2] != 'sil':
            tokens.extend([*line[2]])
            tokens.append(' ')
    return [char_to_int(token) for token in tokens]

all_alignments = []
for video_path in df['video_path'].unique():
    datapath = video_path.split('/')[0]
    speaker_path = video_path.split('/')[-1].split('\\')[0]
    vid_path = video_path.split('/')[-1].split('\\')[-1].split('.')[0]
    
    alignment_path = os.path.join(f'{datapath}','alignments',f'{vid_path}.align')
    alignments = load_alignments(alignment_path) 
    all_alignments.append(alignments)


In [9]:
len(all_alignments)

986

In [63]:
# video_path = df['video_path'].iloc[0]
# datapath = video_path.split('/')
# speakert_path = video_path.split('/')[-1].split('\\')[0]
# vid_path = video_path.split('/')[-1].split('\\')[-1].split('.')[0]

In [11]:

label_data = [np.array(label) for label in all_alignments if label]
# Example of shaping your input data and label data for model training
#input_data = np.stack(chunks)  # Ensure that chunks are correctly reshaped and stacked

# Check shapes (important for debugging)
print("Input data shape:", input_data.shape)
print("Number of label sequences:", len(label_data))



Input data shape: (986, 75, 120)
Number of label sequences: 986


In [14]:
class LipReadingDataset(Dataset):
    def __init__(self, input_features, labels):
        """
        Args:
            input_features (numpy array): Input features with shape (984, 1, 75, 80)
            labels (numpy array): Labels with shape (984, 30)
        """
        self.input_features = input_features
        self.labels = labels

    def __len__(self):
        return len(self.input_features)

    def __getitem__(self, idx):
        # Convert numpy arrays to torch tensors
        video_frames = torch.from_numpy(self.input_features[idx]).float() 
        character_labels = torch.from_numpy(self.labels[idx]).long()
        return video_frames, character_labels

In [15]:
input_data.shape

(986, 75, 120)

In [16]:
def collate_fn(batch):
    inputs, targets = zip(*batch)
    input_lengths = torch.tensor([75] * len(inputs), dtype=torch.long)
    target_lengths = torch.tensor([len(target) for target in targets], dtype=torch.long)
    
    inputs_padded = pad_sequence(inputs, batch_first=True)
    targets_padded = pad_sequence(targets, batch_first=True, padding_value=0)
    
    return inputs_padded, input_lengths, targets_padded, target_lengths

dataset = LipReadingDataset(input_data, label_data)
data_loader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)

In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class LipReadingModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes, num_lstm_layers=2, num_gru_layers=1, dropout_rate=0.5):
        super(LipReadingModel, self).__init__()
        self.input_layer = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU()
        )
        self.lstm = nn.LSTM(hidden_dim, hidden_dim, num_layers=num_lstm_layers, batch_first=True, dropout=dropout_rate if num_lstm_layers > 1 else 0)
        
        self.output_layer = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, num_classes)
        )
        
    def forward(self, x):
        #print(x.shape)
        x = self.input_layer(x)
        #print(x.shape)
        x, _ = self.lstm(x)
        x = self.output_layer(x) # ouput: (batch, seq_len, num_classes)
        #print(x.shape)
        x = F.log_softmax(x, dim=2)
        x = x.transpose(0, 1) # for ctc -> (seq_len, batch, num_classes)
        #print(x.shape)        
        return x

vocab = create_vocab()
vocab_len = len(vocab)
num_classes = vocab_len + 1 

In [18]:
vocab

'abcdefghijklmnopqrstuvwxyz123456789 '

In [94]:
vocab_len

36

In [19]:
num_classes

37

In [93]:
import torch
import numpy as np

def train_model(model, data_loader, criterion, optimizer, device, num_epochs=100):
    model.to(device)
    model.train()
    for epoch in range(num_epochs):
        for batch_idx, (inputs, input_lengths, targets, target_lengths) in enumerate(data_loader):
            inputs = inputs.to(device)
            targets = targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            print(f"Output shape: {outputs.shape}")  # Should match [seq_len, batch, num_classes]
            print(f"Target shape: {targets.shape}, Target lengths: {target_lengths}")
            loss = criterion(outputs, targets, input_lengths, target_lengths)
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


model = LipReadingModel(input_dim=120, hidden_dim=256, num_classes=num_classes, num_lstm_layers=2, num_gru_layers=2, dropout_rate=0.5).to(device)
criterion = nn.CTCLoss(blank=num_classes - 1, zero_infinity=True)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


small_dataset = LipReadingDataset(input_data[:10], label_data[:10])
small_data_loader = DataLoader(small_dataset, batch_size=10, shuffle=True, collate_fn=collate_fn)

train_model(model, small_data_loader, criterion, optimizer, device, num_epochs=1000)


Output shape: torch.Size([75, 10, 37])
Target shape: torch.Size([10, 26]), Target lengths: tensor([23, 26, 26, 24, 25, 24, 23, 24, 25, 26])
Epoch 1, Loss: 8.42088794708252
Output shape: torch.Size([75, 10, 37])
Target shape: torch.Size([10, 26]), Target lengths: tensor([23, 24, 23, 26, 26, 25, 25, 24, 24, 26])
Epoch 2, Loss: 8.282856941223145
Output shape: torch.Size([75, 10, 37])
Target shape: torch.Size([10, 26]), Target lengths: tensor([23, 24, 25, 24, 25, 26, 23, 24, 26, 26])
Epoch 3, Loss: 8.12097454071045
Output shape: torch.Size([75, 10, 37])
Target shape: torch.Size([10, 26]), Target lengths: tensor([26, 24, 26, 24, 25, 24, 26, 23, 25, 23])
Epoch 4, Loss: 7.826539516448975
Output shape: torch.Size([75, 10, 37])
Target shape: torch.Size([10, 26]), Target lengths: tensor([23, 26, 24, 24, 25, 24, 23, 26, 26, 25])
Epoch 5, Loss: 7.155674934387207
Output shape: torch.Size([75, 10, 37])
Target shape: torch.Size([10, 26]), Target lengths: tensor([26, 25, 26, 24, 23, 25, 23, 24, 26, 24

In [95]:
with torch.no_grad():
    for inputs, _, _, _ in small_data_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        print(outputs[0, :, :])  # Print the logits for the first timestep across the batch

tensor([[-1.9857e+01, -1.9866e+01, -1.4543e-05, -1.9364e+01, -1.9470e+01,
         -1.3664e+01, -2.1294e+01, -2.2770e+01, -1.6619e+01, -1.1309e+01,
         -1.9644e+01, -1.9766e+01, -1.5397e+01, -1.9715e+01, -1.6196e+01,
         -1.5361e+01, -1.5264e+01, -1.9866e+01, -1.5600e+01, -2.1263e+01,
         -1.6420e+01, -1.8451e+01, -1.7478e+01, -2.1409e+01, -2.1801e+01,
         -1.9574e+01, -2.0618e+01, -1.9628e+01, -1.9693e+01, -1.9905e+01,
         -1.9870e+01, -1.9698e+01, -1.9404e+01, -1.9706e+01, -1.9582e+01,
         -1.9735e+01, -1.6968e+01],
        [-2.1155e+01, -2.1271e+01, -4.4107e-06, -2.0601e+01, -2.0727e+01,
         -1.4308e+01, -2.2721e+01, -2.4093e+01, -1.7760e+01, -1.2619e+01,
         -2.0921e+01, -2.1018e+01, -1.5892e+01, -2.0998e+01, -1.7409e+01,
         -1.6355e+01, -1.6284e+01, -2.1161e+01, -1.6608e+01, -2.2674e+01,
         -1.7666e+01, -1.9039e+01, -1.8643e+01, -2.2971e+01, -2.3219e+01,
         -2.0855e+01, -2.2156e+01, -2.0927e+01, -2.0962e+01, -2.1194e+01,
  

In [91]:
for i in range(10):
    print(small_dataset[i][0])  # Print the first few inputs to see if they are different


tensor([[ 5.2829e-01,  7.1285e-01, -3.1539e-02,  ...,  5.7689e-01,
          7.3533e-01,  2.3698e-03],
        [ 5.2958e-01,  7.1125e-01, -3.2879e-02,  ...,  5.7854e-01,
          7.3650e-01,  1.1994e-03],
        [ 5.2997e-01,  7.1201e-01, -3.3049e-02,  ...,  5.7867e-01,
          7.3793e-01,  1.2160e-03],
        ...,
        [ 5.2656e-01,  7.0417e-01, -3.3903e-02,  ...,  5.7596e-01,
          7.3111e-01, -1.1588e-03],
        [ 5.2653e-01,  7.0515e-01, -3.4228e-02,  ...,  5.7636e-01,
          7.3209e-01, -1.0656e-03],
        [ 5.2665e-01,  7.0529e-01, -3.3800e-02,  ...,  5.7612e-01,
          7.3203e-01, -3.5558e-04]])
tensor([[ 0.5058,  0.6727, -0.0275,  ...,  0.5602,  0.6938,  0.0111],
        [ 0.5062,  0.6689, -0.0297,  ...,  0.5603,  0.6924,  0.0097],
        [ 0.5089,  0.6696, -0.0305,  ...,  0.5626,  0.6921,  0.0098],
        ...,
        [ 0.5123,  0.6701, -0.0293,  ...,  0.5660,  0.6899,  0.0129],
        [ 0.5130,  0.6700, -0.0297,  ...,  0.5664,  0.6899,  0.0126],
     

In [85]:
first_batch = next(iter(small_data_loader))
    
first_input, _, labels, _ = first_batch 

print("Shape of the input sample for inference:", first_input.shape)

Shape of the input sample for inference: torch.Size([10, 75, 120])


In [86]:
with torch.no_grad():
    log_probs = model(first_input.to(device))


In [87]:
log_probs.shape

torch.Size([75, 10, 37])

In [88]:
''.join(int_to_char(idx) for idx in labels[1])


'bin blue at r nine soon '

In [90]:
max_indices = [np.argmax(log_prob) for log_prob in torch.exp(log_probs[:,1].cpu())]
''.join(int_to_char(idx) for idx in max_indices)

'bin   blue         aat                       eee                           '

In [150]:
import torch
import itertools

def greedy_decoder(ctc_output, blank_label=0):
    probabilities = torch.exp(ctc_output)
    _, max_indices = torch.max(probabilities, 2)
    indices = max_indices.squeeze(1)  
    decoded_sequence = [index for index, group in itertools.groupby(indices) if index != blank_label]
    return decoded_sequence

decoded_sequence = greedy_decoder(log_probs)
print("Decoded sequence indices:", decoded_sequence)


Decoded sequence indices: [tensor(12, device='cuda:0'), tensor(1, device='cuda:0'), tensor(25, device='cuda:0'), tensor(36, device='cuda:0'), tensor(18, device='cuda:0'), tensor(5, device='cuda:0'), tensor(36, device='cuda:0'), tensor(5, device='cuda:0'), tensor(36, device='cuda:0'), tensor(9, device='cuda:0'), tensor(20, device='cuda:0'), tensor(36, device='cuda:0'), tensor(5, device='cuda:0'), tensor(36, device='cuda:0')]


In [151]:
''.join(int_to_char(idx) for idx in decoded_sequence)

'lay re e it e '